In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn import Transformer
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.metrics import roc_curve, roc_auc_score
from tqdm import tqdm
from transformers import BertModel, BertTokenizer, BertConfig, BertForSequenceClassification
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask import delayed, compute

In [3]:
label= pd.read_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/label.csv')

In [4]:
# 데이터 불러오기
data = pd.read_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data.csv')

# dicharge -2

## 6h data

In [4]:
import pandas as pd
from datetime import timedelta
from tqdm import tqdm

def filter_data_by_label(data_df, label_df):
    filtered_data = []
    # 'charttime'을 datetime 형식으로 변환
    data_df.loc[:, 'charttime'] = pd.to_datetime(data_df['charttime'])

    # 라벨 데이터를 순회
    for label_row in tqdm(label_df.itertuples(index=False), total=len(label_df), desc="Filtering data by label"):
        subject_id = label_row.subject_id
        discharge_date = pd.to_datetime(label_row.discharge)

        # 해당 subject_id의 데이터를 미리 필터링
        subject_data = data_df[data_df['subject_id'] == subject_id]

        # 6시간 간격으로 범위를 설정하여 역순으로 데이터를 필터링
        for i in range(10):
            end_time = discharge_date - timedelta(days=1) - timedelta(hours=i * 6)
            start_time = end_time - timedelta(hours=6)

            # 벡터화된 조건으로 데이터를 필터링
            match = subject_data[(subject_data['charttime'] >= start_time) & (subject_data['charttime'] < end_time)]
            
            if not match.empty:
                itemids = match['itemid'].tolist()  # 현재 구간의 모든 itemid를 리스트로 추출
                formatted_time = start_time.strftime('%Y-%m-%d %H:%M')  # 종료 시간을 'MM-DD HH:MM' 형식으로 포맷

                filtered_data.append({
                    'subject_id': subject_id,
                    'time': formatted_time,  # 구간의 종료 시간을 저장
                    'itemids': itemids       # 해당 구간의 모든 itemid를 리스트로 저장
                })

    # 필터링된 데이터를 DataFrame으로 변환
    filtered_data_df = pd.DataFrame(filtered_data)

    return filtered_data_df


In [5]:
filtered_data_df = filter_data_by_label(data, label)

Filtering data by label: 100%|██████████████| 5012/5012 [15:04<00:00,  5.54it/s]


In [6]:
import ast

def remove_duplicates_and_sort(item_list):
    """
    리스트 내의 중복된 요소를 제거하고, 오름차순으로 정렬된 리스트를 반환합니다.
    문자열 형태로 전달된 경우, 이를 리스트로 변환하여 처리합니다.
    """
    if isinstance(item_list, str):
        # 문자열인 경우 리스트로 변환
        item_list = ast.literal_eval(item_list)
    
    # 중복 제거 및 오름차순 정렬
    return sorted(list(set(item_list)))

filtered_data_df['itemids'] = filtered_data_df['itemids'].apply(remove_duplicates_and_sort)

In [7]:
filtered_data_df.to_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_6h.csv', index = False)

## 12h data

In [5]:
import pandas as pd
from datetime import timedelta
from tqdm import tqdm

def filter_data_by_label(data_df, label_df):
    filtered_data = []
    # 'charttime'을 datetime 형식으로 변환
    data_df.loc[:, 'charttime'] = pd.to_datetime(data_df['charttime'])

    # 라벨 데이터를 순회
    for label_row in tqdm(label_df.itertuples(index=False), total=len(label_df), desc="Filtering data by label"):
        subject_id = label_row.subject_id
        discharge_date = pd.to_datetime(label_row.discharge)

        # 해당 subject_id의 데이터를 미리 필터링
        subject_data = data_df[data_df['subject_id'] == subject_id]

        # 6시간 간격으로 범위를 설정하여 역순으로 데이터를 필터링
        for i in range(10):
            end_time = discharge_date - timedelta(days=1) - timedelta(hours=i * 12)
            start_time = end_time - timedelta(hours=12)

            # 벡터화된 조건으로 데이터를 필터링
            match = subject_data[(subject_data['charttime'] >= start_time) & (subject_data['charttime'] < end_time)]
            
            if not match.empty:
                itemids = match['itemid'].tolist()  # 현재 구간의 모든 itemid를 리스트로 추출
                formatted_time = start_time.strftime('%Y-%m-%d %H:%M')  # 종료 시간을 'MM-DD HH:MM' 형식으로 포맷

                filtered_data.append({
                    'subject_id': subject_id,
                    'time': formatted_time,  # 구간의 종료 시간을 저장
                    'itemids': itemids       # 해당 구간의 모든 itemid를 리스트로 저장
                })

    # 필터링된 데이터를 DataFrame으로 변환
    filtered_data_df = pd.DataFrame(filtered_data)

    return filtered_data_df


In [6]:
filtered_data_df = filter_data_by_label(data, label)

Filtering data by label: 100%|██████████████████████████████████████████████████████████████████████████████| 5012/5012 [15:32<00:00,  5.37it/s]


In [7]:
import ast

def remove_duplicates_and_sort(item_list):
    """
    리스트 내의 중복된 요소를 제거하고, 오름차순으로 정렬된 리스트를 반환합니다.
    문자열 형태로 전달된 경우, 이를 리스트로 변환하여 처리합니다.
    """
    if isinstance(item_list, str):
        # 문자열인 경우 리스트로 변환
        item_list = ast.literal_eval(item_list)
    
    # 중복 제거 및 오름차순 정렬
    return sorted(list(set(item_list)))

filtered_data_df['itemids'] = filtered_data_df['itemids'].apply(remove_duplicates_and_sort)

In [8]:
filtered_data_df.to_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_12h.csv', index = False)

## 24h data

In [9]:
import pandas as pd
from datetime import timedelta
from tqdm import tqdm

def filter_data_by_label(data_df, label_df):
    filtered_data = []
    # 'charttime'을 datetime 형식으로 변환
    data_df.loc[:, 'charttime'] = pd.to_datetime(data_df['charttime'])

    # 라벨 데이터를 순회
    for label_row in tqdm(label_df.itertuples(index=False), total=len(label_df), desc="Filtering data by label"):
        subject_id = label_row.subject_id
        discharge_date = pd.to_datetime(label_row.discharge)

        # 해당 subject_id의 데이터를 미리 필터링
        subject_data = data_df[data_df['subject_id'] == subject_id]

        # 6시간 간격으로 범위를 설정하여 역순으로 데이터를 필터링
        for i in range(10):
            end_time = discharge_date - timedelta(days=1) - timedelta(hours=i * 24)
            start_time = end_time - timedelta(hours=24)

            # 벡터화된 조건으로 데이터를 필터링
            match = subject_data[(subject_data['charttime'] >= start_time) & (subject_data['charttime'] < end_time)]
            
            if not match.empty:
                itemids = match['itemid'].tolist()  # 현재 구간의 모든 itemid를 리스트로 추출
                formatted_time = start_time.strftime('%Y-%m-%d %H:%M')  # 종료 시간을 'MM-DD HH:MM' 형식으로 포맷

                filtered_data.append({
                    'subject_id': subject_id,
                    'time': formatted_time,  # 구간의 종료 시간을 저장
                    'itemids': itemids       # 해당 구간의 모든 itemid를 리스트로 저장
                })

    # 필터링된 데이터를 DataFrame으로 변환
    filtered_data_df = pd.DataFrame(filtered_data)

    return filtered_data_df


In [10]:
filtered_data_df = filter_data_by_label(data, label)

Filtering data by label: 100%|██████████████████████████████████████████████████████████████████████████████| 5012/5012 [15:20<00:00,  5.45it/s]


In [11]:
import ast

def remove_duplicates_and_sort(item_list):
    """
    리스트 내의 중복된 요소를 제거하고, 오름차순으로 정렬된 리스트를 반환합니다.
    문자열 형태로 전달된 경우, 이를 리스트로 변환하여 처리합니다.
    """
    if isinstance(item_list, str):
        # 문자열인 경우 리스트로 변환
        item_list = ast.literal_eval(item_list)
    
    # 중복 제거 및 오름차순 정렬
    return sorted(list(set(item_list)))

filtered_data_df['itemids'] = filtered_data_df['itemids'].apply(remove_duplicates_and_sort)

In [12]:
filtered_data_df.to_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_24h.csv', index = False)

## 48h data

In [13]:
import pandas as pd
from datetime import timedelta
from tqdm import tqdm

def filter_data_by_label(data_df, label_df):
    filtered_data = []
    # 'charttime'을 datetime 형식으로 변환
    data_df.loc[:, 'charttime'] = pd.to_datetime(data_df['charttime'])

    # 라벨 데이터를 순회
    for _, label_row in tqdm(label_df.iterrows(), total=len(label_df), desc="Filtering data by label"):
        subject_id = label_row['subject_id']
        discharge_date = pd.to_datetime(label_row['discharge'])

        # 해당 subject_id의 데이터를 필터링
        subject_data = data_df[data_df['subject_id'] == subject_id]

        # 24시간 간격으로 범위를 설정하여 역순으로 데이터를 필터링
        for i in range(10):
            end_time = discharge_date - timedelta(days=1) - timedelta(hours=i * 48)
            start_time = end_time - timedelta(hours=48)

            # start_time과 end_time 사이의 데이터를 필터링
            match = subject_data[(subject_data['charttime'] >= start_time) & (subject_data['charttime'] < end_time)]
            
            if not match.empty:
                itemids = match['itemid'].tolist()  # 현재 구간의 모든 itemid를 리스트로 추출
                formatted_time = start_time.strftime('%Y-%m-%d %H:%M')  # 종료 시간을 'MM-DD HH:MM' 형식으로 포맷

                filtered_data.append({
                    'subject_id': subject_id,
                    'time': formatted_time,  # 구간의 종료 시간을 저장
                    'itemids': itemids       # 해당 구간의 모든 itemid를 리스트로 저장
                })

    # 필터링된 데이터를 DataFrame으로 변환
    filtered_data_df = pd.DataFrame(filtered_data)

    return filtered_data_df


In [14]:
filtered_data_df = filter_data_by_label(data, label)

Filtering data by label: 100%|██████████████████████████████████████████████████████████████████████████████| 5012/5012 [15:16<00:00,  5.47it/s]


In [15]:
import ast

def remove_duplicates_and_sort(item_list):
    """
    리스트 내의 중복된 요소를 제거하고, 오름차순으로 정렬된 리스트를 반환합니다.
    문자열 형태로 전달된 경우, 이를 리스트로 변환하여 처리합니다.
    """
    if isinstance(item_list, str):
        # 문자열인 경우 리스트로 변환
        item_list = ast.literal_eval(item_list)
    
    # 중복 제거 및 오름차순 정렬
    return sorted(list(set(item_list)))

filtered_data_df['itemids'] = filtered_data_df['itemids'].apply(remove_duplicates_and_sort)

In [16]:
filtered_data_df.to_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_48h.csv', index = False)